### In the section below I scraped NBA player of the week award from the wonderful site 'basketball.realgm.com'

In [103]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

player_of_the_week_data = {
 'Last Season': [],  
 'Player': [],
 'Team': [],
 'Conference': [],
 'Date': [],
 'Position': [],
 'Height': [],
 'Weight': [],
 'Age': [],
 'Draft Year': [],
 'Seasons in league': [],
 'Season': [],
 'Season short': [],
 'Pre-draft Team': []
}


In [40]:
player_of_the_week_url = 'https://basketball.realgm.com/nba/awards/by-type/Player-Of-The-Week/30'

page = requests.get(player_of_the_week_url)
soup = BeautifulSoup(page.text, 'html.parser')

In [41]:
tables = soup.find_all('table')
rows = tables[12].findAll('tr')[1:]

In [109]:
from_year = 1979
to_year = 2999

for row in rows:
    
    cols = row.findAll('td')
    season = cols[0].string
    season_short = int(season[5:])
    if season_short > from_year and season_short <= to_year:
        player_of_the_week_data['Last Season'].append(0)
        player_of_the_week_data['Player'].append(cols[1].string)
        player_of_the_week_data['Team'].append(cols[4].text)
        player_of_the_week_data['Conference'].append(cols[2].string)
        player_of_the_week_data['Date'].append(cols[3].string)
        player_of_the_week_data['Position'].append(cols[5].string)
        player_of_the_week_data['Height'].append(cols[6].string.replace('-',"'"))
        player_of_the_week_data['Weight'].append(cols[7].text)
        player_of_the_week_data['Age'].append(cols[8].string)
        player_of_the_week_data['Draft Year'].append(cols[10].string)
        player_of_the_week_data['Seasons in league'].append(cols[11].string)
        player_of_the_week_data['Season'].append(season)
        player_of_the_week_data['Season short'].append(season_short)
        player_of_the_week_data['Pre-draft Team'].append(cols[9].string)

In [110]:
df_player_of_the_week = pd.DataFrame(player_of_the_week_data)


# Assign last Year
max_season = df_player_of_the_week['Season'].max()
df_player_of_the_week['Last Season'] = df_player_of_the_week['Season'].apply(lambda x: 1 if x == max_season else 0)


# if two awards given [east & west] on the same week the player gets 0.5 point,
#,else gets 1 point
df_player_of_the_week["Real_value"] = df_player_of_the_week.apply(lambda pl: 0.5 if 
                                                                  pl["Conference"] =="East" or pl["Conference"] == "West" else 1,axis=1)


# weight pounds to KGs
df_player_of_the_week['Height CM'] = df_player_of_the_week['Height'].apply(lambda x:
                                                                           round(((float(x.split("'")[0]) * 12) + float(x.split("'")[1])) * 2.54))
df_player_of_the_week['Weight KG'] = df_player_of_the_week['Weight'].apply(lambda x: int(int(x) * 0.454))


df_player_of_the_week.to_csv('NBA_player_of_the_week.csv', index=False)